# Week 4: Data transformation, cleaning and loading with Python

## WaterInfo Data Set
We are considering a water data set this week about Murray River Basin in NSW. In Canvas (Module 4 section), there is an Excel sheet available with an excerpt of the original data set. We also have made available the content of the Excel workbook as a set of four different CSV files. Please upload those CSV files to Jupyter first. 

**Important:** Make sure that the naming of all the files is as follows:
 1. Measurements.csv
 2. Organisations.csv
 3. Sensors.csv
 4. Stations.csv

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 2: Data Loading and Database Creation with Python

Next we are back to Python. We continue with the same Python environment than last week: the `DictReader` from the `csv` module which support reading and writing of files in comma-separated values (CSV).

Make sure that you have uploaded the 'Organisations.csv' CSV file into Jupyter.
We will first load the content of this file into Python with the same  csv.DictReader()  mechanism than last week:

In [1]:
import csv
import pprint
data_organisations = list(csv.DictReader(open('Organisations.csv')))
pprint.pprint(data_organisations[0])

OrderedDict([('Code', 'DNR'),
             ('Organisation',
              'NSW Department of Water and Energy (and predecessors)')])


For larger data sets, the following would normally be executed as a stand alone Python program on a shell.
First, you need to establish a connection to the postgresql database. 
__Please edit the unikey and SID variables in below's code to match your Jupyter login.__

In [2]:
import psycopg2

def pgconnect():
    # please replace with your own details
    YOUR_DBNAME = 'comp5310'    
    YOUR_USERNAME = 'postgres'
    YOUR_PW     = '277698'
    try: 
        conn = psycopg2.connect(host='localhost',
                                database=YOUR_DBNAME,
                                user=YOUR_USERNAME, 
                                password=YOUR_PW)
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

We will need to execute some SQL statements against the database. As we will have to do so multiple times, we write a dedicated function for executing an arbitrary SQL statement, where we do not expect any result. This handles then also all failures and using psycopg2's 'with' statements also the transaction processing of the database. Below's code will for example automatically commit our SQL statements, as well as rollback if there was any error.

In [3]:
def pgexec( conn, sqlcmd, args, msg, silent=False ):
   """ utility function to execute some SQL statement
       can take optional arguments to fill in (dictionary)
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
               cur.execute(sqlcmd)
            else:
               cur.execute(sqlcmd, args)
            if silent == False: 
                print("success: " + msg)
            retval = True
         except Exception as e:
            if silent == False: 
                print("db error: ")
                print(e)
   return retval


Now let's load our previous data.
Important: whenever you use this approach, make sure that the header line of your CSV file has no spaces in its column titles and also no quotes. Otherwise, the csv.DictReader might be fine to read it, but not the psycopg2's cursor.execute() function. We are using named placeholders in out INSERT statement below (eg. '%(SiteName)s' ) which expects to put a string (%s) into that place of the INSERT statement as been found in the given dictionary for the execute() call with the key 'SiteName'.

In [4]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS Organisation", None, "Reset Table Organisation")

# 2nd: ensure that the schema is in place

organisation_schema = """CREATE TABLE IF NOT EXISTS Organisation (
                         code VARCHAR(20) PRIMARY KEY,
                         orgName VARCHAR(150)
                   )"""
pgexec (conn, organisation_schema, None, "Create Table Organisation")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Organisation(code,orgName) VALUES (%(Code)s, %(Organisation)s)"""
for row in data_organisations:
    pgexec (conn, insert_stmt, row, "row inserted")



connected
success: Reset Table Organisation
success: Create Table Organisation
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted


Next let's check whether this has all worked fine by querying our PostgreSQL database again.
You of course can go back to the Terminal page and in pgsql simply type   SELECT * FROM Organisation

Or we do it here in Python again. To do so, we introduce first another utility function which again encapsulates all error and transaction handling. Then we query the new Organisation table and simply print out all tuples found.

In [5]:
def pgquery( conn, sqlcmd, args, silent=False ):
   """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
                cur.execute(sqlcmd)
            else:
                cur.execute(sqlcmd, args)
            if silent == False:
                for record in cur:
                    print(record)
            retval = True
         except Exception as e:
            if silent == False:
                print("db read error: ")
                print(e)
   return retval

In [6]:

# check content of Organisation table
query_stmt = "SELECT * FROM Organisation"
print(query_stmt)
pgquery (conn, query_stmt, None)

# cleanup...   Needed already?  Better not now... 
# But keep in mind to close connection eventually!
# conn.close()

SELECT * FROM Organisation
('DNR', 'NSW Department of Water and Energy (and predecessors)')
('DWR', 'NSW Department of Water and Energy (and predecessors)')
('MIL', 'Murray Irrigation Ltd')
('PWD', 'Manly Hydraulics Laboratory')
('QWR', 'Qld Department of Natural Resources and Water')
('SCA', 'Sydney Catchment Authority')
('SMA', 'Snowy Mountains Authority')
('SWB', 'Sydney Catchment Authority')
('VRW', 'Vic Government')


True

## Your Task: Data Loading

Try to create and load the Measurement table.

    1.read the Measurements csv file
    2. Create a matching 'Measurement' table to hold the CSV data
    3. Load the content of the csv file into a local 'data_measurements' dictionary in Python
    4. Load the data from the 'data_measurements' dictionary into your PostgreSQL table
    5. Query and print its content

In [7]:
# TODO: replace the content of this cell with your Python + psycopg2 solution
#raise NotImplementedError

##1st: read the Measurements csv file

data_measurements = list(csv.DictReader(open('Measurements.csv')))
pprint.pprint(data_measurements[0])



# 2nd: ensure that the schema is in place
pgexec (conn, "DROP TABLE IF EXISTS Measurement", None, "Reset Table Measurement")
measurement_schema = """CREATE TABLE IF NOT EXISTS Measurement (
                         station   VARCHAR(20),
                         date DATE,
                         level  FLOAT,
                         meanDischarge FLOAT,
                         discharge  FLOAT,
                         temp FLOAT,
                         ec  FLOAT
                      )"""
pgexec (conn, measurement_schema, None, "Create Table Measurement")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Measurement(station,date,level,meandischarge,discharge,temp,ec)
                      VALUES (%(Station)s, %(Date)s, %(Level)s,%(MeanDischarge)s,%(Discharge)s,%(Temp)s,%(EC)s)"""
# 4th: insert the data into the table
for row in data_measurements:
    pgexec (conn, insert_stmt, row, "row inserted")
   
#5th: Query  content of Measurement table
query_stmt = "SELECT * FROM Measurement"
print(query_stmt)
pgquery (conn, query_stmt, None)

query_stmt = "SELECT count(*) FROM Measurement"
print(query_stmt)
pgquery (conn, query_stmt, None)


OrderedDict([('Station', '409204C'),
             ('Date', '31-Dec-04'),
             ('Level', '1.196'),
             ('MeanDischarge', '5876.087'),
             ('Discharge', '5873.837'),
             ('Temp', '23.217'),
             ('EC', '57.583')])
success: Reset Table Measurement
success: Create Table Measurement
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...UES ('219018', '31-Dec-04', '0.219','1.996','1.731','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...LUES ('219018', '1-Jan-05', '0.208','1.406','1.273','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...LUES ('219018', '2-Jan-05', '0.201','1.096','1.166','x','x')
                               

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...    VALUES ('219018', '14-Feb-05', '-0.248','0','0','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...    VALUES ('219018', '15-Feb-05', '-0.247','0','0','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...    VALUES ('219018', '16-Feb-05', '-0.243','0','0','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...    VALUES ('219018', '17-Feb-05', '-0.206','0','0','x','x')
                                                               ^

success: row inserte

db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...    VALUES ('219018', '31-Mar-05', '-0.173','0','0','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...     VALUES ('219018', '1-Apr-05', '-0.098','0','0','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...       VALUES ('219018', '2-Apr-05', '0.04','0','0','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...LUES ('219018', '3-Apr-05', '0.166','0.198','0.371','x','x')
                                                               ^

success: row inserted
success: row inserted
db error: 
invalid i


success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...                   VALUES ('219018', '16-May-05', 'x','x','x...
                                                             ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...                   VALUES ('219018', '17-May-05', 'x','x','x...
                                                             ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...                   VALUES ('219018', '18-May-05', 'x','x','x...
                                                             ^

success: row inserted
success: row inserted
db error: 
invalid input syntax for type double precision: "x"
LINE 2: ...                   VALUES ('219018', '19-May-05', 'x','x','x...
                                                             ^

success: row in

True

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 2: Data Cleaning

### Data Cleaning
We re-use the clean() function from last week.

In [8]:
import numpy as np
DEFAULT_VALUE = np.nan

def clean(data, column_key, convert_function, default_value):
    special_values= {} # no special values yet
    for row in data:
        old_value = row[column_key]
        new_value = default_value
        try:
            if old_value in special_values.keys():
                new_value = special_values[old_value]
            else:
                new_value = convert_function(old_value)
        except (ValueError, TypeError):
            print('Replacing {} with {} in column {}'.format(row[column_key], new_value, column_key))
        row[column_key] = new_value
             

In [9]:
# this conversion strips any leading or trailing spaces from the 'Station' values
clean(data_measurements, 'Station', str.strip, DEFAULT_VALUE)

# the following converts the two measurment  columns to float  values - or NaN
clean(data_measurements, 'Discharge', float, DEFAULT_VALUE)
clean(data_measurements, 'MeanDischarge', float, DEFAULT_VALUE)
clean(data_measurements, 'Level', float, DEFAULT_VALUE)
clean(data_measurements, 'Temp', float, DEFAULT_VALUE)
clean(data_measurements, 'EC', float, DEFAULT_VALUE)

##now we insert the  data_measurements into the 'Measurement' table 
# 1st: login to database
if(conn):
    conn.close();
conn = pgconnect()

# 2nd: ensure that the schema is in place
pgexec (conn, "DROP TABLE IF EXISTS Measurement", None, "Reset Table Measurement")
measurement_schema = """CREATE TABLE IF NOT EXISTS Measurement (
                         station   VARCHAR(20),
                         date DATE,
                         level  FLOAT,
                         meanDischarge FLOAT,
                         discharge  FLOAT,
                         temp FLOAT,
                         ec  FLOAT
                      )"""
pgexec (conn, measurement_schema, None, "Create Table Measurement")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Measurement(station,date,level,meandischarge,discharge,temp,ec)
                      VALUES (%(Station)s, %(Date)s, %(Level)s,%(MeanDischarge)s,%(Discharge)s,%(Temp)s,%(EC)s)"""
for row in data_measurements:
    pgexec (conn, insert_stmt, row, "row inserted")
    
query_stmt = "SELECT COUNT(*) FROM Measurement"
print(query_stmt)
pgquery (conn, query_stmt, None)

Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column Discharge
Replacing x with nan in column MeanDischarge
Replacing x with nan in column MeanDischarge
Replacing x with nan in column MeanDischarge
Replacing x with nan in column MeanDischarge
Replacing x with nan in column MeanDischarge
Replacing x with nan in column MeanDi

success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: r

True

## Your Task: Data Cleaning

Use above's  clean()  function to clean the other give data set too.
 1. read the Stations csv file into data_stations
 2. Clean the  'data_stations'  data set
 3. Reload the 'data_stations'  dictionary into your database
 4. Query the 'Stations' table - which difference do you see?
 
 5. If you have time: Do all of the above (reading - cleaning - loading) also for the 'Sensors.csv' data set

Note: You might encounter a few warning and error messages.
   - If a connection is closed, you have to open the databse connection again first
   - If the clean() function returns a warning that some string was replaced with NaN, as long as this is indeed a number attribute, you are Ok to ignore this message. It just tells you that it is doing what it is supposed to do.
   - If you try to insert data into an already existing table with data inside, you might get 'duplicate primary key' error messages. Again, you can ignore those for the moment.
   - If you want to see who much data is already in your table, use the following SQL query:
     -  SELECT COUNT(*) FROM Stations;

In [11]:
# TODO: replace the content of this cell with your Python solution
#raise NotImplementedError


data_stations = list(csv.DictReader(open('Stations.csv')))
pprint.pprint(data_stations[0])

# this conversion strips any leading or trailing spaces from the 'Station' values
clean(data_stations, 'BasinNo', str.strip, DEFAULT_VALUE)
clean(data_stations, 'Site', str.strip, DEFAULT_VALUE)
clean(data_stations, 'Commence', str.strip, DEFAULT_VALUE)


##now we inser the  data_stations into the 'Stations' table 
# 1st: login to database
if(conn):
    conn.close();
conn = pgconnect()

# 2nd: ensure that the schema is in place
pgexec (conn, "DROP TABLE IF EXISTS Station", None, "Reset Table Station")
stations_schema = """CREATE TABLE IF NOT EXISTS Station (
                         basinNO   VARCHAR(50),
                         site VARCHAR(50),
                         siteName  VARCHAR(50),
                         lon VARCHAR(50),
                         lat  VARCHAR(50),
                         commence Date,
                         orgCode  VARCHAR(50)                         
                      )"""
pgexec (conn, stations_schema, None, "Create Table Station")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Station(basinNO,site,siteName,lon,lat,commence,orgCode)
                      VALUES (%(BasinNo)s, %(Site)s, %(SiteName)s,%(Long)s,%(Lat)s,%(Commence)s,%(OrgCode)s)"""
for row in data_stations:
    pgexec (conn, insert_stmt, row, "row inserted")

query_stmt = "SELECT * FROM Station"
print(query_stmt)
pgquery (conn, query_stmt, None)

query_stmt = "SELECT COUNT(*) FROM Station"
print(query_stmt)
pgquery (conn, query_stmt, None)

   
    
###Sensors data

data_sensors = list(csv.DictReader(open('Sensors.csv')))
pprint.pprint(data_sensors[0])


##now we insert the  data_stations into the 'Stations' table 
# 1st: login to database
if(conn):
    conn.close();
conn = pgconnect()

# 2nd: ensure that the schema is in place
pgexec (conn, "DROP TABLE IF EXISTS Sensor", None, "Reset Table Sensor")
stations_schema = """CREATE TABLE IF NOT EXISTS Sensor (
                         sensor   VARCHAR(20) PRIMARY KEY,
                         description  VARCHAR(150),
                         metric  VARCHAR(150)                   
                      )"""
pgexec (conn, stations_schema, None, "Create Table Sensor")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Sensor(sensor,description,metric)
                      VALUES (%(sensor)s, %(description)s, %(metric)s)"""
for row in data_sensors:
    pgexec (conn, insert_stmt, row, "row inserted")

query_stmt = "SELECT COUNT(*) FROM Sensor"
print(query_stmt)
pgquery (conn, query_stmt, None)

   
  


OrderedDict([('BasinNo', '409'),
             ('Site', '001'),
             ('SiteName', 'Murray River at Albury (Union Bridge)'),
             ('Long', '146.8957 E'),
             ('Lat', '36.0929 S'),
             ('Commence', '14/04/1892'),
             ('OrgCode', 'DWR')])
connected
success: Reset Table Station
success: Create Table Station
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
SELECT * FROM Station
('409', '001', 'Murray River at Albury (Union Bridge)', '146.8957 E', '36.0929 S', datetime.date(1892, 4, 14), 'DWR')
('409', '002', 'Murray River at Corowa', '146.3954 E', '36.0076 S', datetime.date(1894, 4, 1), 'DWR')
('409', '003', 'Murray River at Denuquin', '144.9663 E', '35.5301 S', datetime.date(1896, 9, 1), 'DWR')
('409', '005', 'Murray River at Barham', '144.1235 E', '35.6304 S', datetime.date(1900, 12, 31), 'DWR')
('409

True

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## Demo: Data Modelling & DB Creation
Next, we will look in the lecture at modelling the data correctly for a normalised relational star schema. We will discuss an example solution for the given water data model in the class.

## Your Homework Task: Data Modelling
Have a look at the Excel workbook and check how we have modeled its content into a normalised relational data model as shown in our proposed relational schema further down in this notebook.

# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 3: Database Creation
After we discussed the model, we will give you an example solution (see below).
The next step is to create the corresponding SQL schema in your PostgreSQL database.

### Your Task: DB Creation in PostgreSQL
Create the corresponding tables in PostgreSQL which follow from the data model.

<pre>
DROP TABLE IF EXISTS Organisation CASCADE;
CREATE TABLE IF NOT EXISTS Organisation (
   code VARCHAR(20) PRIMARY KEY,
   organisation    VARCHAR(150)
);

DROP TABLE IF EXISTS Station CASCADE;
CREATE TABLE IF NOT EXISTS Station (
     station   VARCHAR(50) PRIMARY KEY,
     siteName  VARCHAR(50),
     commence Date,
     orgCode  VARCHAR(50),
     CONSTRAINT orgCodeFK FOREIGN KEY (orgCode)   REFERENCES Organisation (code)
 );

DROP TABLE IF EXISTS Sensor CASCADE;
CREATE TABLE IF NOT EXISTS Sensor (
     sensor   VARCHAR(20) PRIMARY KEY,
     description  VARCHAR(150) ,       
     metric VARCHAR(20)                 
  );

DROP TABLE IF EXISTS Measurement CASCADE;
CREATE TABLE IF NOT EXISTS Measurement (
     station   VARCHAR(20),
     sensor   VARCHAR(20),
     date DATE,
     value  FLOAT,
     CONSTRAINT stationFK FOREIGN KEY (station)   REFERENCES Station (Station),
     CONSTRAINT sensorFK FOREIGN KEY (sensor)   REFERENCES sensor (sensor)
  );
                      
</pre>

In [12]:
# TODO: replace the content of this cell with your Python + SQL solution
##raise NotImplementedError
# make sure we are still connected to database 
if conn is None or conn.closed:
    conn = pgconnect()
    
table_schema = """
DROP TABLE IF EXISTS Organisation CASCADE;
CREATE TABLE IF NOT EXISTS Organisation (
   code VARCHAR(20) PRIMARY KEY,
   organisation    VARCHAR(150)
);

DROP TABLE IF EXISTS Station CASCADE;
CREATE TABLE IF NOT EXISTS Station (
     station   VARCHAR(50) PRIMARY KEY,
     siteName  VARCHAR(50),
     commence Date,
     orgCode  VARCHAR(50),
     CONSTRAINT orgCodeFK FOREIGN KEY (orgCode)   REFERENCES Organisation (code)
 );

DROP TABLE IF EXISTS Sensor CASCADE;
CREATE TABLE IF NOT EXISTS Sensor (
     sensor   VARCHAR(20) PRIMARY KEY,
     description  VARCHAR(150) ,       
     metric VARCHAR(20)                 
  );

DROP TABLE IF EXISTS Measurement CASCADE;
CREATE TABLE IF NOT EXISTS Measurement (
     station   VARCHAR(20),
     date DATE,
     sensor   VARCHAR(20),
     value  FLOAT,
     CONSTRAINT stationFK FOREIGN KEY (station)   REFERENCES Station (station),
     CONSTRAINT sensorFK FOREIGN KEY (sensor)   REFERENCES sensor (sensor)
  );"""   

pgexec (conn, table_schema, None, "Created New Schema")

conn.close();

success: Created New Schema


# STOP PLEASE. THE FOLLOWING IS FOR THE NEXT EXERCISE. THANKS.

## EXERCISE 4: Data Loading and Storage

Up-to this point, we have
 - analysed and modelled the given data set
 - created a corresponding relational star schema
 - read the individual CSV files into Python dictionary data structures
 - cleaned the raw data with regard to missing or inconsistent entries and data types
 
The final step is to load this cleaned data into the corresponding tables of the star schema which we defined so far.

For this to work, you probably will need to write some logic to load different parts of different data dictionaries (holding the content of CSV files) into different tables.


In [13]:
import numpy as np
import csv
import pprint
# Organisation table
data_organisations = list(csv.DictReader(open('Organisations.csv')))

# make sure we are still connected to database 
if conn is None or conn.closed:
    conn = pgconnect()

# check for any existing content of the Organisations table
query_stmt = "SELECT COUNT(*) FROM Organisation"
print(query_stmt)
pgquery (conn, query_stmt, None)

# Try to load data - 9 tupels should be created
insert_stmt = """INSERT INTO Organisation(code,organisation) VALUES (%(Code)s, %(Organisation)s)"""
for row in data_organisations:
    pgexec (conn, insert_stmt, row, "row inserted")
    
#####################
# Station table    
data_stations = list(csv.DictReader(open('Stations.csv')))

# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Station(station,siteName,commence,orgCode)
                      VALUES (%(station)s, %(siteName)s, %(commence)s,%(orgCode)s)"""
stationData = dict()
for row in data_stations:
    stationData['station'] = row['BasinNo']+row['Site']
    stationData['siteName']  = row['SiteName']
    stationData['commence'] = row['Commence']
    stationData['orgCode'] = row['OrgCode']
    pgexec (conn, insert_stmt, stationData, "station inserted")

# check content of Station table
query_stmt = "SELECT * FROM Station"
print(query_stmt)
pgquery (conn, query_stmt, None)



#####################
# Sensor table    
data_sensors = list(csv.DictReader(open('Sensors.csv')))

# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Sensor(sensor,description,metric)
                      VALUES (%(sensor)s, %(description)s, %(metric)s)"""

for row in data_sensors:
    pgexec (conn, insert_stmt, row, "sensor inserted")

# check content of Station table
query_stmt = "SELECT * FROM Sensor"
print(query_stmt)
pgquery (conn, query_stmt, None)


#####################
# Measurement table    
data_measurements = list(csv.DictReader(open('Measurements.csv')))

# the following converts the two measurment  columns to float  values - or NaN
clean(data_measurements, 'Discharge', float, DEFAULT_VALUE)
clean(data_measurements, 'MeanDischarge', float, DEFAULT_VALUE)
clean(data_measurements, 'Level', float, DEFAULT_VALUE)
clean(data_measurements, 'Temp', float, DEFAULT_VALUE)
clean(data_measurements, 'EC', float, DEFAULT_VALUE)

# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Measurement(station,date,sensor,value)
                      VALUES (%(station)s, %(date)s, %(sensor)s, %(value)s)"""

measurementData = dict()
sensorCodes = ['levl', 'disvol','disc','temp', 'ec']
sensor_columns = ['Level', 'MeanDischarge', 'Discharge','Temp', 'EC']
for row in data_measurements:
    measurementData['station'] = row['Station']
    measurementData['date'] = row['Date']
    for i in range(len(sensorCodes)):
        if(np.isnan(row[sensor_columns[i]])):
            continue;
        measurementData['sensor'] = sensorCodes[i]
        measurementData['value']  = row[sensor_columns[i]]
        pgexec (conn, insert_stmt, measurementData, "measurement inserted")
    

# check content of Measurement table
query_stmt = "SELECT * FROM Measurement"
print(query_stmt)
pgquery (conn, query_stmt, None)

query_stmt = "SELECT count(*) FROM Measurement"
print(query_stmt)
pgquery (conn, query_stmt, None)
conn.close();

connected
SELECT COUNT(*) FROM Organisation
(0,)
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: station inserted
success: station inserted
success: station inserted
success: station inserted
success: station inserted
success: station inserted
success: station inserted
success: station inserted
success: station inserted
SELECT * FROM Station
('409001', 'Murray River at Albury (Union Bridge)', datetime.date(1892, 4, 14), 'DWR')
('409002', 'Murray River at Corowa', datetime.date(1894, 4, 1), 'DWR')
('409003', 'Murray River at Denuquin', datetime.date(1896, 9, 1), 'DWR')
('409005', 'Murray River at Barham', datetime.date(1900, 12, 31), 'DWR')
('409204C', 'Murray River @ Swan Hill', datetime.date(1904, 12, 31), 'VRW')
('409017', 'Murray River @ Doctors Point', datetime.date(1925, 8, 22), 'DWR')
('409019', 'Wakool River at Offtake Reg

success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: m

success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: m

success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: m

success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: m

success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: measurement inserted
success: m

('219018', datetime.date(2004, 12, 31), 'levl', 0.219)
('219018', datetime.date(2004, 12, 31), 'disvol', 1.996)
('219018', datetime.date(2004, 12, 31), 'disc', 1.731)
('409017', datetime.date(2004, 12, 31), 'levl', 2.552)
('409017', datetime.date(2004, 12, 31), 'disvol', 9615.444)
('409017', datetime.date(2004, 12, 31), 'disc', 9689.209)
('409017', datetime.date(2004, 12, 31), 'temp', 21.072)
('409017', datetime.date(2004, 12, 31), 'ec', 46.104)
('409204C', datetime.date(2005, 1, 1), 'levl', 1.19)
('409204C', datetime.date(2005, 1, 1), 'disvol', 5835.842)
('409204C', datetime.date(2005, 1, 1), 'disc', 5795.7)
('409204C', datetime.date(2005, 1, 1), 'temp', 23.167)
('409204C', datetime.date(2005, 1, 1), 'ec', 58.0)
('219018', datetime.date(2005, 1, 1), 'levl', 0.208)
('219018', datetime.date(2005, 1, 1), 'disvol', 1.406)
('219018', datetime.date(2005, 1, 1), 'disc', 1.273)
('409017', datetime.date(2005, 1, 1), 'levl', 2.558)
('409017', datetime.date(2005, 1, 1), 'disvol', 9694.446)
('409

# End of Tutorial. Many Thanks.